# Problem
In this report i will try optimizing the answer for the traveling salesman problem(TSP) using genetic algorithms.
A genetic algorithm is a search heuristic that is inspired by Charles Darwin's theory of natural evolution. This algorithm reflects the process of natural selection where the fittest individuals are selected for reproduction in order to produce offspring of the next generation. Now each genetic algorithm has some important functions which I will explain how they work in my code in this report.


# Getting genes and their original fitness
in the test case that we had (bayg29) the matrix for 29 cities was given to us which had the distance between each city. This distance is also our fitness table, which means for each route that is a permutation of the 29 cities , we have to sum the distance between each to adjacent city.
First we have to import the 2 libraries that are needed and define the basic parameters:


In [1]:
import pandas as pd
import random

start = 0
length = 29
bridge_weight = {}
city_list = [0]
route_list = []
best_length = []
best_routes = []

Then we will read our test case and create our bridge_list which is the distance between each 2 cities that will be used for generating fitness.

In [2]:
f = open("bayg29.tsp", "r")
txt = f.readlines()
for i in range(1,29):
    city_list.append(i)
for i in range (8,36):
    weight = [float(j) for j in txt[i].split()]
    for w in range(len(weight)):
        bridge_weight[start, w+start+1] = weight[w]
        bridge_weight[w+start+1, w] = weight[w]
    start += 1


Now we have to create our first generation which will be derived from random permutations of our cities and the number of these random results is based on the population we want in our code.

In [3]:
population = 1000
for i in range(population):
    route_list.append(random.sample(city_list,29))

df = {"routes" : route_list}
df = pd.DataFrame(df)


# Fitness
For this part we will calculate the fitness of each of our routes and then sort our dataframe so that the best fitness will be the first row of our dataframe. The reason we do this part is that when we want to pick the next generation, it is best to not create the children randomly from each parent and choose the parents wisely.

In [4]:
def calculate_fitness(route):
    distance = 0
    # print(route)
    for i in range(len(route) - 1):
        distance += bridge_weight[route[i+1], route[i]]
    distance += bridge_weight[route[-1], route[0]]
    return 1 / float(distance)
    
def fitness_sort(df, new_gen):
    for i in range(len(df)):
        df.loc[i, "fitness"] = calculate_fitness(df.loc[i, "routes"])
    if(new_gen == True):
        max = df["fitness"].idxmax()
        best_length.append(1 / df.loc[max, 'fitness'])
        best_routes.append(df.loc[max, 'routes'])
    df = df.sort_values(by=["fitness"],ascending=False)
    df = df.reset_index()
    result = []
    for i in range(len(df)):
        result.append(df.loc[i, 'routes'])
    df = pd.DataFrame({'routes' : result})
    return df

# Breed
In this part we have to make the childrens of this generation. Several algorithms were presented for doing this part but I found that simply cutting the parents is the best way possible for a gene like this. But for better results I found it best to choose the breaking point randomly. This will allow the algorithm to create a more realistic Genetic Algorithm. But since when cutting the parent and choosing each part from one of them it is possible to have a repeated city in a route we have to check and if that happens ignore the created child.

In [5]:
def breed(parentA, parentB):
    child = []
    break_point = random.randint(0, len(parentA)-1)
    for i in range(len(parentA)):
        if i <= break_point:
            child.append(parentA[i])
        else:
            child.append(parentB[i])
    for i in range(len(child)):
        for j in range(i+1, len(child)):
            if child[i] == child[j]:
                return None
    return child

# Mutate
As we learned in class it is more realistic to mutate some of the childs so that not every child is a copy of their represented parents .This will allow the algorithm to be more natural. For this goal we have different algorithms and between the ones that are suitable for this program I picked the Swap Mutation which is easier and more efficient in my point of view.

In [6]:
def mutate(child):
    x = random.randint(0, len(child)-1)
    y = random.randint(0, len(child)-1)
    first_gene = child[x]
    second_gene = child[y]
    child[y] = first_gene
    child[x] = second_gene
    return child

# New generation
Now we have to create our next generation. After getting the result of each generation we have to create the next to see if we can get a better result in the new generation. For this part the solution I used was to randomly decide how many childs we want to create and then replace the new childs with the parents that had the worst fitness. Also the children's parents will be chosen from the best half of the parents.(parents with best fitness)

In [7]:
def next_generation(df):
    childs = []
    new_childs_number = random.randint(0, len(df)-1)
    df = fitness_sort(df, False)
    new_route_list = []
    while(len(childs)< new_childs_number):
        a_parent_indx = random.randint(0, len(df)//2)
        b_parent_indx = random.randint(0, len(df)//2)
        result = breed(df.loc[a_parent_indx, 'routes'], df.loc[b_parent_indx, 'routes'])
        if result:
            childs.append(result)
    mutate_number = random.randint(0, new_childs_number)
    for i in range(mutate_number):
        child_indx = random.randint(0, len(childs)-1)
        childs[child_indx] = mutate(childs[child_indx])
    replace = 0
    for i in range(len(df)-1,0, -1): 
        if replace < len(childs):
            new_route_list.append(childs[replace])
            replace += 1
        else:
            new_route_list.append(df.loc[i, 'routes'])
    df = pd.DataFrame({'routes' : new_route_list})
    return df

# TSP-algorithm
Finally we will calculate the fitness of each generation and find the best fitness(least distance) after the given number of generations.

In [8]:
def TSP_genetic_algorithm(df):
    count = 0
    while(count< 1000):
        count += 1
        df = fitness_sort(df, True)
        df = next_generation(df)
    print(min(best_length))
    print(best_routes[best_length.index(min(best_length))])
    return

TSP_genetic_algorithm(df)

1940.0
[10, 14, 22, 6, 11, 8, 15, 12, 7, 21, 16, 3, 17, 13, 24, 18, 20, 5, 9, 19, 27, 23, 28, 25, 4, 1, 0, 2, 26]


# Interesting facts
While doing this project and trying to get the best result possible, I found out that creating a bigger population and more generations can result in a better response but will also take more time to run. Of Course increasing these numbers isn't always helpful and will sometimes generate worse results. The best numbers I found That resulted in the least distance for me were the ones I used in this report.
